In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import os
# Navigate to a specific folder
dataset_folder = '/content/drive/My Drive/dataset2/'
dataset_files = os.listdir(dataset_folder)

# Display the list of files in that folder
print(dataset_files)

['spotify_millsongdata.csv']


In [ ]:
train_dir = "/content/drive/My Drive/dataset2/spotify_millsongdata.csv" #passing the path with training imagess

In [ ]:
df = pd.read_csv(train_dir)

In [ ]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [ ]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Patti Smith,Paths That Cross,Speak to me \r\nSpeak to me heart \r\nI feel...
1,John Waite,I-95,In the cruel month grey morning \r\nI woke up...
2,Queen Adreena,Wolverines,on my back i lay it down for plenty \r\nthese...
3,Misfits,Wolfs Blood,You don't know what I've spoken to \r\nWhile ...
4,Pat Benatar,The Victim,You've been set up one too many times \r\nThe...
5,Iron Maiden,Coming Home,When I stand before you shining \r\nIn the ea...
6,Ozzy Osbourne,Waiting For Darkness,Waiting for darkness \r\nDoesn't anybody see ...
7,Rod Stewart,Just Like A Woman,"Nobody feels any pain, \r\nTonight as I stand..."
8,John Prine,You Got Gold,Is there ever enough space between us \r\nTo ...
9,Verve,Velvet Morning,Yes \r\nIt's been long \r\nAnd yes \r\nI st...


In [ ]:
df['text'][0]

"Speak to me  \r\nSpeak to me heart  \r\nI feel a needing  \r\nTo bridge the clouds  \r\nSoftly go  \r\nA way I wish to know  \r\nA way I wish to know  \r\n  \r\nOh you'll ride  \r\nSurely dance  \r\nIn a ring  \r\nBackwards and forwards  \r\nThose who seek  \r\nFeel the glow  \r\nA glow we will all know  \r\nA glow we will all know  \r\n  \r\nOn that day  \r\nFilled with grace  \r\nAnd the heart's communion  \r\nSteps we take  \r\nSteps we trace  \r\nInto the light of reunion  \r\n  \r\nPaths that cross  \r\nWill cross again  \r\nPaths that cross  \r\nWill cross again  \r\n  \r\nSpeak to me  \r\nSpeak to me shadow  \r\nI spin from the wheel  \r\nNothing at all  \r\nSave the need  \r\nThe need to weave  \r\nA silk of souls  \r\nThat whisper whisper  \r\nA silk of souls  \r\nThat whispers to me  \r\n  \r\nSpeak to me heart  \r\nAll things renew  \r\nHearts will mend  \r\nRound the bend  \r\nPaths that cross  \r\nCross again  \r\nPaths that cross  \r\nWill cross again  \r\n  \r\nRise up 

In [ ]:
df.shape

(5000, 3)

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

array([1.        , 0.01567477, 0.03087766, ..., 0.00394636, 0.00204848,
       0.01839626])

In [ ]:
df[df['song'] == 'Paths That Cross']

,artist,song,text
0,Patti Smith,Paths That Cross,speak to me speak to me heart i feel a need to...


In [ ]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if df[df['song'] == song_df].empty:
        return "Song not found in the dataset."  # Return a message if song is not found
    else:
        idx = df[df['song'] == song_df].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)
        return songs

In [ ]:
recommendation('Paths That Cross')

['Worthless Lie',
 'Speak To Me',
 'Crosses',
 "Don't Cross The River",
 'Many Rivers To Cross',
 'Crucified With Christ',
 'Out Of Time',
 'At The Foot Of The Cross (Ashes To Beauty)',
 'Sabotage',
 'Supernaut',
 'Bottom Of Your Soul',
 'The Gift',
 "I'll Be Your San Antonio Rose",
 'Forbidden Zone',
 'Bye Bye (Jay-Z Remix)',
 'In Our Sleep',
 'Always',
 'Cross-Eyed Mary',
 'Lift High The Cross',
 'The Glow Of Love']

In [ ]:
from google.colab import files
files.download('similarity.pkl')
files.download('df.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>